In [ ]:
use catalog identifier(:catalog);
use schema identifier(:schema);

In [0]:

CREATE OR REPLACE PROCEDURE get_clustering_metadata(
  metadata_catalog_name STRING,
  metadata_schema_name STRING DEFAULT NULL,
  metadata_table_name STRING DEFAULT NULL
)
COMMENT 'Gets table metadata from information_schema with optional filtering by catalog, schema, and table name. Makes HTTP requests to get additional metadata from Databricks API.

Parameters:
- metadata_catalog_name (mandatory): The catalog name to filter by
- metadata_schema_name (optional): The schema name to filter by
- metadata_table_name (optional): The table name to filter by'
SQL SECURITY INVOKER
LANGUAGE SQL
AS
BEGIN
  SELECT
    table_catalog,
    table_schema,
    table_name,
    table_type,
    resp.data_source_format,
    CASE
      WHEN resp.properties.clusteringColumns IS NOT NULL THEN
        from_json(resp.properties.clusteringColumns, 'array<array<string>>')
      ELSE NULL
    END AS clustering_columns
  FROM (
    SELECT
      table_catalog,
      table_schema,
      table_name,
      table_type,
      get_table(table_catalog, table_schema, table_name) as resp
    FROM IDENTIFIER(metadata_catalog_name || '.information_schema.tables')
    WHERE table_catalog = metadata_catalog_name
      AND (metadata_schema_name IS NULL OR table_schema = metadata_schema_name)
      AND (metadata_table_name IS NULL OR table_name = metadata_table_name)
  ) t;
END;